In [16]:
import pandas as pd
import numpy as np

df = pd.read_csv("Hitters.csv")
df = df.copy()
df.head()
#veri setimizde index problemi yok 


,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [17]:
df.shape


(322, 20)

In [18]:
df.isnull().sum()
# salary değişkenlerimizde boş gözlemler bulunmaktadır 
# bunları ortalama ile dolduralım 


AtBat         0
Hits          0
HmRun         0
Runs          0
RBI           0
Walks         0
Years         0
CAtBat        0
CHits         0
CHmRun        0
CRuns         0
CRBI          0
CWalks        0
League        0
Division      0
PutOuts       0
Assists       0
Errors        0
Salary       59
NewLeague     0
dtype: int64

In [19]:
df["Salary"].fillna(df.Salary.mean() , inplace = True)
# boş maaş değerlerini ortalama ile doldurduk aslında bunları ilerde tahmin ile dolduracağız

In [20]:
df.isnull().sum().any()
# datamızda boş değer var mı ? 


False

In [21]:
df.info()
# veri setimizdeki kategorik değişkenleri düzenliyelim 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 20 columns):
AtBat        322 non-null int64
Hits         322 non-null int64
HmRun        322 non-null int64
Runs         322 non-null int64
RBI          322 non-null int64
Walks        322 non-null int64
Years        322 non-null int64
CAtBat       322 non-null int64
CHits        322 non-null int64
CHmRun       322 non-null int64
CRuns        322 non-null int64
CRBI         322 non-null int64
CWalks       322 non-null int64
League       322 non-null object
Division     322 non-null object
PutOuts      322 non-null int64
Assists      322 non-null int64
Errors       322 non-null int64
Salary       322 non-null float64
NewLeague    322 non-null object
dtypes: float64(1), int64(16), object(3)
memory usage: 50.4+ KB


# Kategorik değişkenleri modellemeye hazır hale getirme 

In [22]:
from sklearn.preprocessing import LabelEncoder 
cat_list = ["NewLeague" , "Division" , "League"] 

lbe = LabelEncoder()
for i in cat_list:
    df[i] = lbe.fit_transform(df[i])

In [23]:
df.head(3)
# tip dönüşümlerini başarılı bir şekilde uyguladık 

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,0,0,446,33,20,535.925882,0
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,1,1,632,43,10,475.000000,1
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,0,1,880,82,14,480.000000,0


# Bagimli ve bağımsız değişken ayrışımı 

In [24]:
X = df.drop("Salary"  , axis = 1) # bagımsız degisken 

In [25]:
y = df["Salary"] #bagimli değisken 

# Veri setini Train_Test olarak ayırma

In [26]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.20)

In [27]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
# doğru parcalamışız

(257, 19)
(257,)
(65, 19)
(65,)


# Model Kurma

In [28]:
from sklearn.linear_model import ElasticNet

In [29]:
enet = ElasticNet(alpha= 1.2,
                  l1_ratio = 0.5,
                 random_state = 1).fit(X_train , y_train)
# l1_ratio değeri ;
     # 0 olur ise l2 cezalandırması (Ridge)
    # 1 olur ise l1 cezalandorması yapar(Lasso)
    # 0.5 ikisindende aynı etkide yapar 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10992395.379080057, tolerance: 4055.832786948735
  positive)


In [30]:
enet.coef_

array([ -1.57970546,   2.05105445,  -0.74214259,   3.78536904,
         2.75700111,   1.67249202,  -8.76653616,  -0.42949013,
         1.70507511,   1.22679897,   0.6203799 ,  -0.45414635,
        -0.16830033,   7.56664259, -17.89492603,   0.28445296,
         0.35925915,  -5.11170151,   3.45593191])

## tahmin 

In [35]:
from sklearn.metrics import mean_squared_error ,r2_score

In [36]:
y_pred = enet.predict(X_test)

In [37]:
np.sqrt( mean_squared_error( y_test , y_pred ) )

384.2378137221795

In [38]:
r2_score( y_test ,y_pred )

0.23386093497880378

# Tuning 

In [39]:
from sklearn.linear_model import ElasticNetCV 

In [40]:
enet_cross = ElasticNetCV(alphas = None,
                        cv = 10,
                         random_state = 0).fit(X_train , y_train)


In [42]:
optimum_alp = enet_cross.alpha_
optimum_alp

816.2988050492321

## Final modeli kurmak 

In [44]:
enet_final = ElasticNet(alpha = optimum_alp ,random_state = 1).fit(X_train ,y_train)

In [46]:
y_pred = enet_final.predict(X_test)

In [47]:
np.sqrt( mean_squared_error(y_test , y_pred) )

390.6881873406929